# Introduction to JavaScript

Introduce JavaScript, its importance, and its role in web development.

> This curriculum and code was initiatlly created using GitHub Copilot Chat using this prompt:
> `Create a curriculum for teaching JavaScript for beginners. Provide instructions in Markdown and code snippets in JavaScript` 

## OpenAI Usage

Here is a simple example of usage to validate that we have OpenAI integration working in the Notebook. This tests a simple chat completion


In [ ]:
import { OpenAI } from "openai";

// OpenAI-Node is the official library for JS/TS
// See: https://github.com/openai/openai-node

// Install It:
// Add `npm --save openai` 👉🏽 for post-create-update script for container

// Configure Env Vars:
// gets API Key from environment variable OPENAI_API_KEY
// setup to use dotenv => accessed as process.env.OPENAI_API_KEY
//     For Codespaces => Add into GitHub Codespaces secrets
//     For Docker Desktop => Set them in .env
const openai = new OpenAI({
    organization: process.env.OPENAI_ORG_ID,
    apiKey: process.env.OPENAI_API_KEY,
});

// List of Models Available:
//    https://platform.openai.com/docs/models/overview
// List of Models I can Access:
//    Call API with your API-key to determine this
// gpt-4 Access is currently limited
//    https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4
//
// const useModel='gpt-4';
const useModel='gpt-3.5-turbo';

// Demo Example: Function Call
// https://github.com/openai/openai-node/blob/master/examples/demo.ts
async function main() {

  // List models:
  const list = await openai.models.list();
  console.log("✨✨✨ Models I can use and fine tune:")
  console.log(list);

  // Non-streaming:
  console.log("✨✨✨ Non-streaming example:")
  const completion = await openai.chat.completions.create({
    model: useModel,
    messages: [{ role: 'user', content: 'Say this is a test' }],
  });
  console.log(completion.choices[0]?.message?.content);

  // Streaming: 
  console.log("✨✨✨ Streaming example:")
  const stream = await openai.chat.completions.create({
    model: useModel,
    messages: [{ role: 'user', content: 'Say this is a test' }],
    stream: true,
  });
  for await (const part of stream) {
    process.stdout.write(part.choices[0]?.delta?.content || '');
  }
  process.stdout.write('\n');
}

main();

In [ ]:
// There are many other examples of OpenAI usage from the Node.js library here:
// https://github.com/openai/openai-node/tree/master/examples
//
// Example: #TODO
// Validate usage with Azure Open AI
// https://github.com/openai/openai-node/blob/master/examples/azure.ts
//
// Example: Function Calls
// https://github.com/openai/openai-node/blob/master/examples/function-call.ts

import OpenAI from 'openai';
import { ChatCompletionMessage, ChatCompletionMessageParam } from 'openai/resources/chat';

// gets API Key from environment variable OPENAI_API_KEY
const openai = new OpenAI();
const useModel='gpt-3.5-turbo';

const functions: OpenAI.Chat.ChatCompletionCreateParams.Function[] = [
  {
    name: 'list',
    description: 'list queries books by genre, and returns a list of names of books',
    parameters: {
      type: 'object',
      properties: {
        genre: { type: 'string', enum: ['mystery', 'nonfiction', 'memoir', 'romance', 'historical'] },
      },
    },
  },
  {
    name: 'search',
    description: 'search queries books by their name and returns a list of book names and their ids',
    parameters: {
      type: 'object',
      properties: {
        name: { type: 'string' },
      },
    },
  },
  {
    name: 'get',
    description:
      "get returns a book's detailed information based on the id of the book. Note that this does not accept names, and only IDs, which you can get by using search.",
    parameters: {
      type: 'object',
      properties: {
        id: { type: 'string' },
      },
    },
  },
];

async function callFunction(function_call: ChatCompletionMessage.FunctionCall): Promise<any> {
  const args = JSON.parse(function_call.arguments!);
  switch (function_call.name) {
    case 'list':
      return await list(args['genre']);

    case 'search':
      return await search(args['name']);

    case 'get':
      return await get(args['id']);

    default:
      throw new Error('No function found');
  }
}

async function main() {
  
  const messages: ChatCompletionMessageParam[] = [
    {
      role: 'system',
      content: 'Please use our book database, which you can access using functions to answer the following questions.',
    },
    {
      role: 'user',
      content: 'I really enjoyed reading To Kill a Mockingbird, could you recommend me a book that is similar and tell me why?',
    },
  ];
  console.log(messages[0]);
  console.log(messages[1]);
  console.log();

  
  while (true) {
    const completion = await openai.chat.completions.create({
      model: useModel,
      messages,
      functions: functions,
    });

    const message = completion.choices[0]!.message;
    messages.push(message);
    console.log(message);

    // If there is no function call, we're done and can exit this loop
    if (!message.function_call) {
      console.log("✨✨✨  No function call found. Exiting. ✨✨✨");
      return;
    }

    // If there is a function call, we generate a new message with the role 'function'.
    const result = await callFunction(message.function_call);
    const newMessage = {
      role: 'function' as const,
      name: message.function_call.name!,
      content: JSON.stringify(result),
    };
    messages.push(newMessage);

    console.log("✨✨✨ There was a function call. Pushing New Message. ✨✨✨");
    console.log(newMessage);
    console.log();
  }
}

const db = [
  {
    id: 'a1',
    name: 'To Kill a Mockingbird',
    genre: 'historical',
    description: `Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.`,
  },
  {
    id: 'a2',
    name: 'All the Light We Cannot See',
    genre: 'historical',
    description: `In a mining town in Germany, Werner Pfennig, an orphan, grows up with his younger sister, enchanted by a crude radio they find that brings them news and stories from places they have never seen or imagined. Werner becomes an expert at building and fixing these crucial new instruments and is enlisted to use his talent to track down the resistance. Deftly interweaving the lives of Marie-Laure and Werner, Doerr illuminates the ways, against all odds, people try to be good to one another.`,
  },
  {
    id: 'a3',
    name: 'Where the Crawdads Sing',
    genre: 'historical',
    description: `For years, rumors of the “Marsh Girl” haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.

But Kya is not what they say. A born naturalist with just one day of school, she takes life's lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world—until the unthinkable happens.`,
  },
];

async function list(genre: string) {
  return db.filter((item) => item.genre === genre).map((item) => ({ name: item.name, id: item.id }));
}

async function search(name: string) {
  return db.filter((item) => item.name.includes(name)).map((item) => ({ name: item.name, id: item.id }));
}

async function get(id: string) {
  return db.find((item) => item.id === id)!;
}

main();


## Variables

In JavaScript, variables are declared using the `var` keyword. They can hold any type of value, including numbers, strings, booleans, arrays, and objects. Variables can be assigned a value using the assignment operator `=`. Here are some examples:

In [ ]:

var x = 5; // number
console.log("x is: " + x);
var name = "John"; // string
console.log("name is: " + name);
var isTrue = true; // boolean
console.log("isTrue is: " + isTrue);
var fruits = ["apple", "banana", "orange"]; // array
console.log("fruits is: " + fruits);
var person = {name: "John", age: 30}; // object
console.log("person is: " + person);


## Data Types

JavaScript has several built-in data types, including:

- Number: represents numeric values.
- String: represents textual data.
- Boolean: represents either `true` or `false`.
- Array: represents an ordered list of values.
- Object: represents a collection of key-value pairs.

Here are some examples:


In [ ]:

var age = 25; // Number
var name = "John"; // String
var isTrue = true; // Boolean
var fruits = ["apple", "banana", "orange"]; // Array
var person = {name: "John", age: 30}; // Object



## Operators

JavaScript supports various operators for performing operations on values. Some common operators include:

- Arithmetic operators: `+`, `-`, `*`, `/`, `%`.
- Comparison operators: `==`, `===`, `!=`, `!==`, `>`, `<`, `>=`, `<=`.
- Logical operators: `&&`, `||`, `!`.

Here are some examples:


In [ ]:

var x = 5;
var y = 10;
var sum = x + y; // Addition
var difference = x - y; // Subtraction
var product = x * y; // Multiplication
var quotient = x / y; // Division
var remainder = x % y; // Modulo

var isEqual = x == y; // Equality
var isGreater = x > y; // Greater than
var isLess = x < y; // Less than

var isTrue = true;
var isFalse = !isTrue; // Negation
var isBothTrue = isTrue && isTrue; // Logical AND
var isEitherTrue = isTrue || isFalse; // Logical OR



## Control Flow

JavaScript provides control flow statements for executing different blocks of code based on certain conditions. Some common control flow statements include:

- If statement: executes a block of code if a specified condition is true.
- If-else statement: executes a block of code if a specified condition is true, otherwise executes a different block of code.
- Switch statement: selects one of many code blocks to be executed.
- For loop: repeats a block of code a specified number of times.
- While loop: repeats a block of code while a specified condition is true.

Here are some examples:


In [ ]:

var x = 5;

if (x > 0) {
  console.log("x is positive");
} else if (x < 0) {
  console.log("x is negative");
} else {
  console.log("x is zero");
}

var day = "Monday";

switch (day) {
  case "Monday":
    console.log("It's Monday");
    break;
  case "Tuesday":
    console.log("It's Tuesday");
    break;
  default:
    console.log("It's another day");
    break;
}

for (var i = 0; i < 5; i++) {
  console.log("Iteration " + i);
}

var i = 0;
while (i < 5) {
  console.log("Iteration " + i);
  i++;
}



## Functions

Functions are reusable blocks of code that perform a specific task. They can accept input parameters and return a value. Here's an example of a function:


In [ ]:

function greet(name) {
  console.log("Hello, " + name + "!");
}

greet("John"); // Output: Hello, John!



## Arrays

Arrays are used to store multiple values in a single variable. They can be accessed and manipulated using index values. Here's an example of an array:


In [ ]:

var fruits = ["apple", "banana", "orange"];

console.log(fruits[0]); // Output: apple
console.log(fruits.length); // Output: 3

fruits.push("grape"); // Add an element to the end of the array
console.log(fruits); // Output: ["apple", "banana", "orange", "grape"]

fruits.pop(); // Remove the last element from the array
console.log(fruits); // Output: ["apple", "banana", "orange"]



## Objects

Objects are used to store collections of key-value pairs. Each key is a string, and each value can be of any data type. Here's an example of an object:


In [ ]:

var person = {
  name: "John",
  age: 30,
  city: "New York"
};

console.log(person.name); // Output: John
console.log(person.age); // Output: 30
console.log(person.city); // Output: New York

person.name = "Jane"; // Update the value of a property
console.log(person.name); // Output: Jane

delete person.age; // Delete a property
console.log(person); // Output: { name: "Jane", city: "New York" }


This curriculum covers the basics of JavaScript, including variables, data types, operators, control flow, functions, arrays, and objects. It provides a solid foundation for beginners to start learning JavaScript programming. Feel free to modify and expand upon this curriculum based on your teaching needs. Happy coding!